In [ ]:
# This is the application. You can run it.

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from bs4 import BeautifulSoup
from selenium import webdriver
from IPython.core.display import HTML
from selenium import webdriver
import datetime
from datetime import date, timedelta
from time import sleep
from collections import defaultdict
import re
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import requests


In [14]:
#The thing that is really missing is to set the url as a parameter.
#If you want to run the app, do not forget to have selenium installed and to change the  post message parameters for your ifftt account

def check_flights(start_day, nbr_of_ranges, day_delta, eps_val, min_diff_price):
    #start_day is the day when this algorithm should start searching for price, it must be entered as follow : "YYYY-MM-DD"
    #nbr_of_ranges is the number of pages you want the algo to scrap
    #day_delta is the interval of days between between two pages. For example, if start_day is monday 2020-01-01 and day_delta is 7, then the algo with scrap every monday until it reaches the number of range (nbr_of_ranges)
    #min_diff_price : this is a parameter to avoid fare mistakes. It is the minium difference of price between the fare and the closest price. For short-distance trip, I think 30€ is ok. For long and expensive, you can go until 100€. Up to you of course.
    #eps_val : this is linked to the clustering algo. if you do not know it, put 1.05, it will work most of the times
    
    #start_day = '2020-01-20' #this should be a parameter
    start_day_date = datetime.datetime.strptime(start_day, '%Y-%m-%d')
    fare_dict = dict()

    for i in range(nbr_of_ranges):  #this should be a parameter  
        day_start = str(start_day_date).split()[0]
        fare_dict.update({day_start: {}})
        sats = "https://www.google.com/flights?hl=en#flt=/m/06gmr./m/01ly5m." + \
        day_start + ";c:EUR;e:1;sd:1;t:f;tt:o"
        sleep(np.random.randint(2,5))
        driver = webdriver.Chrome()
        driver.get(sats)
        try:
            price = driver.find_element_by_css_selector('.gws-flights-results__cheapest-price').text
            duration = driver.find_element_by_css_selector('.gws-flights-results__duration').text
            stops = driver.find_element_by_css_selector('.gws-flights-results__itinerary-stops').text
            flight_compagny = driver.find_element_by_css_selector('.gws-flights-results__carriers').text
        except:
            None
        fare_dict[day_start] = price, duration, stops, flight_compagny
        df1 = pd.DataFrame.from_dict(fare_dict, orient='index', columns=['price', 'duration', 'stops', 'flight_compagny'])
        start_day_date = start_day_date + timedelta(days=day_delta) #this should be a parameter
        driver.close()
        
    df1['price']= df1['price'].str.extract('(\d+)', expand=False)
    df1['price'] = df1['price'].astype(float)
    
    px = [x for x in df1['price']]
    ff = pd.DataFrame(px, columns=['fare']).reset_index()
    X = StandardScaler().fit_transform(ff)
    db = DBSCAN(eps=eps_val, min_samples=1).fit(X)

    labels = db.labels_
    clusters = len(set(labels))
    unique_labels = set(labels)
    colors = plt.cm.Spectral(np.linspace(0,
                            1, len(unique_labels)))

    plt.subplots(figsize=(12,8))

    for k, c in zip(unique_labels, colors):
        class_member_mask = (labels == k)
        xy = X[class_member_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o',
                 markerfacecolor=c,
                 markeredgecolor='k',
                 markersize=14)

    plt.title("Total Clusters: {}".format(clusters),
              fontsize=14, y=1.01)
    pf = pd.concat([ff,pd.DataFrame(db.labels_,
                                columns=['cluster'])], axis=1)
    rf = pf.groupby('cluster')['fare'].agg(['min','count']).sort_values('min', ascending=True)
    print(rf)
    if clusters > 1\
    and ff['fare'].min() == rf.iloc[0]['min']\
    and rf.iloc[0]['count'] <= rf['count'].quantile(.10)\
    and rf.iloc[0]['min'] + min_diff_price < rf.iloc[1]['min']: #this should be a parameter
        
        city = "Paris-Rio"
        fare = ff['fare'].min()
        requests.post('https://maker.ifttt.com/trigger/fare_alert/with/key/gS5SGXZ_JKP9IS4nNJ0K3T5bh5Zgt0zhzqjCyTaV5Ta', data={"value1": city, "value2": fare})
    else:
        print(rf.iloc[0]['count'])
        print(rf['count'].quantile(.10))
        print('no alert triggered')       

In [ ]:
check_flights('2020-05-09', 10, 1, 1.05, 30)

NameError: name 'px' is not defined

In [6]:
#If you want it to run every hour dot that : 
schedule.every(60).minutes.do(check_flights('2020-01-30', 2, 1, 1.05, 30))

NameError: name 'df1' is not defined